<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
#!pip3 install folium
#!pip3 install wget
# !pip install folium==0.8.3

In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
#spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

#Read Data from CSV
spacex_df=pd.read_csv("spacex_launch_geo.csv")

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns, remove unnecessary columns
spacex_df = spacex_df[["Launch Site", "Lat", "Long", "class"]]

# Combine CCAFS LC-40 and CCAFS SLC-40 into CCAFS, rename KSC LC-39A to KSC, and VAFB SLC-4E to VAFB
spacex_df["Launch Site"] = spacex_df["Launch Site"].replace(
    {"CCAFS LC-40": "CCAFS", "CCAFS SLC-40": "CCAFS", "KSC LC-39A": "KSC", "VAFB SLC-4E":"VAFB"})

# Fix KSC Launch site lat and long 
spacex_df.loc[spacex_df["Launch Site"] == "KSC", "Lat"] = 28.608389
spacex_df.loc[spacex_df["Launch Site"] == "KSC", "Long"] = -80.604333

#group the df by launch site and rename columns
Launch_Sites_df = spacex_df.groupby(["Launch Site"], as_index=False).first()

# Add Long form Launch Site Name
Launch_Sites_df["Launch_Site_Long_Form"] = ["Cape Canaveral Space Launch Complex",
                                        "Kennedy Space Center Launch Complex",
                                        "Vandenberg Space Launch Complex"]

# rename some columns from Launch_Sites_df
cols_to_rename = {"Lat": "Launch_Site_Lat", "Long": "Launch_Site_Long", "Launch Site":"Launch_Site_Abbreviation"}
Launch_Sites_df = Launch_Sites_df.rename(columns=cols_to_rename)

#Drop "class" column from Launch_Sites_df
Launch_Sites_df = Launch_Sites_df.drop(columns=["class"])

#add empty columns to Launch_Sites_df
new_columns = ["Closest_Coastline", "Closest_City", "Closest_Railway", "Closest_Highway", "Launch_Site_Lat",
                "Launch_Site_Long", "Coastline_Lat", "Coastline_Long", "City_Lat", "City_Lat", "City_Long",
                "Railway_Lat", "Railway_Long", "Highway_Lat", "Highway_Long"]
Launch_Sites_df[new_columns] = None

#Launch_Sites_df

Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
Start_location = [29.559684888503615, -95.0830971930759] # NASA Johnson Space Center

# Initialize the map
site_map = folium.Map(location=Start_location, zoom_start = 4)

In [7]:
# Define a dictionary to store the coordinates for each location type
coordinates = {
    "Launch_Site": {"Lat": [28.562302, 28.608389, 34.632834], "Long": [-80.577356, -80.604333, -120.610745]},
    "Coastline": {"Lat": [28.576994, 28.63994, 34.63149], "Long": [-80.51157, -80.54866, -120.69957]},
    "City": {"Lat": [28.40585, 28.60623, 34.646111], "Long": [-80.58986, -80.80942, -120.460278]},
    "Railway": {"Lat": [28.57211, 28.6118, 34.62179], "Long": [-80.58529, -80.5981, -120.63092]},
    "Highway": {"Lat": [28.359850, 28.501658, 34.713189], "Long": [-80.794684, -80.838687, -120.489079]}
}
# Iterate over the dictionary and add the latitude and longitude columns to the dataframe
for key, value in coordinates.items():
    Launch_Sites_df[f"{key}_Lat"] = value["Lat"]
    Launch_Sites_df[f"{key}_Long"] = value["Long"]

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center"s coordinate with a popup label showing its name
coordinate = [29.559684888503615, -95.0830971930759] # NASA Johnson Space Center
circle = folium.Circle(coordinate, radius=1000, color="#0000ff",fill=True).add_child(folium.Popup("NASA Johnson Space Center"))
site_map.add_child(circle);

# Create a blue circle at NASA Johnson Space Center"s coordinate with a icon showing its name
marker = folium.map.Marker(
    coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        #rotation = 45,
        icon_size=(300, 20),# width, height
        icon_anchor=(-10, 0),# +Left, +up
        html="<div style=\"font-size: 12; color: cadetblue;\"><b>%s</b></div>" % "NASA Johnson Space Center",
    )
)
site_map.add_child(marker);

#site_map

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values.
#   In addition, add Launch site name as a popup label

#for i, row in Launch_Sites_df.iterrows():
for i in range(0, len(Launch_Sites_df)):
    coordinate = [Launch_Sites_df.loc[i, "Launch_Site_Lat"], Launch_Sites_df.loc[i, "Launch_Site_Long"]]
    
    circle = folium.Circle(coordinate, radius=1000, color="#0000ff",fill=True).add_child(folium.Popup(Launch_Sites_df.loc[i,"Launch_Site_Long_Form"]))
    site_map.add_child(circle);
    
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            #rotation = 45,
            icon_size=(300, 20), # width, height
            icon_anchor=(-10, 0), # +Left, +up
        html="<div style=\"font-size: 12; color:cadetblue;\"><b>%s</b></div>" % Launch_Sites_df.loc[i,"Launch_Site_Long_Form"]))
    site_map.add_child(marker);
    
#site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
    # Yes, they are all in proximity of the Equator line.
*   Are all launch sites in very close proximity to the coast?
    # Yes, they are all in proximity of the coast.


Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [10]:
Launch_Sites_df.tail(10)

,Launch_Site_Abbreviation,Launch_Site_Lat,Launch_Site_Long,Launch_Site_Long_Form,Closest_Coastline,Closest_City,Closest_Railway,Closest_Highway,Coastline_Lat,Coastline_Long,City_Lat,City_Long,Railway_Lat,Railway_Long,Highway_Lat,Highway_Long
0,CCAFS,28.562302,-80.577356,Cape Canaveral Space Launch Complex,None,None,None,None,28.576994,-80.51157,28.405850,-80.589860,28.57211,-80.58529,28.359850,-80.794684
1,KSC,28.608389,-80.604333,Kennedy Space Center Launch Complex,None,None,None,None,28.639940,-80.54866,28.606230,-80.809420,28.61180,-80.59810,28.501658,-80.838687
2,VAFB,34.632834,-120.610745,Vandenberg Space Launch Complex,None,None,None,None,34.631490,-120.69957,34.646111,-120.460278,34.62179,-120.63092,34.713189,-120.489079


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [11]:
# Create MarkerCluster Object
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [12]:
# Apply a function to check the value of `class` column
# # If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [13]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return "green"
    else:
        return "red"

spacex_df["marker_color"] = spacex_df["class"].apply(assign_marker_color)
#spacex_df.tail()

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
# Create and add a Marker cluster to the site map
for index, row in spacex_df.iterrows():  
    coordinate = [row["Lat"], row["Long"]]
    folium.map.Marker(coordinate, icon=folium.Icon(
        color="white", icon_color=row["marker_color"])).add_to(marker_cluster) #lightblue

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster);

#site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position="topright",
    separator=" Long: ",
    empty_string="NaN",
    lng_first=False,
    num_digits=20,
    prefix="Lat:",
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position);

#site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [16]:
# Function to calculate distance
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [17]:
# Create a marker (in km) with distance to a closest coastline, city, railway, and highway.
def calculate_distance_and_create_marker(row, location_type, color):
    temp_distance = 0
    temp_distance = calculate_distance(row["Launch_Site_Lat"], row["Launch_Site_Long"],
                                        row[location_type + "_Lat"], row[location_type + "_Long"])
    row["Closest_" + location_type] = temp_distance

    distance_marker = folium.Marker(
        [row[location_type + "_Lat"], row[location_type + "_Long"]],
        icon=DivIcon(
            icon_size=(100, 10), # width, height
            icon_anchor=(0, 0), # +Left, +up
            html="<div style=\"font-size: 12; color:{};\"><b>{:10.2f} km</b></div>".format(color, temp_distance),
        )
    )
    site_map.add_child(distance_marker)
    return row

location_types = ["Coastline", "City", "Railway", "Highway"]
colors = ["blue", "red", "green", "purple"]

for location_type, color in zip(location_types, colors):
    Launch_Sites_df = Launch_Sites_df.apply(calculate_distance_and_create_marker, location_type=location_type, color=color, axis=1)

#Launch_Sites_df

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [18]:
# Create Polylines to a closest coastline, city, railway, and highway.

def create_polyline(row, location_type, color):
    coordinates = [[row["Launch_Site_Lat"], row["Launch_Site_Long"]],
                    [row[location_type + "_Lat"], row[location_type + "_Long"]]]
    line = folium.PolyLine(locations=coordinates, weight=2, color=color)
    site_map.add_child(line)

location_types = ["Coastline", "City", "Railway", "Highway"]
colors = ["blue", "red", "green", "purple"]

for location_type, color in zip(location_types, colors):
    Launch_Sites_df.apply(create_polyline, location_type=location_type, color=color, axis=1)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


In [19]:
# Save Map to File
site_map.save("Folium Map of Launch Cites With Proximity Lines.html")

#Save Data to File
Launch_Sites_df.to_csv("Launch_Sites_df", index=False)
spacex_df.to_csv("spacex_df_after", index=False)

In [20]:
#site_map
#Launch_Sites_df
#spacex_df

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
        #Yes, average distance to railway is 1.16 km away, likely due to shipping concerns 

*   Are launch sites in close proximity to highways?
        #Yes, average distance to highway is 4.05 km away, likely due to shipping concerns 

*   Are launch sites in close proximity to coastline?
        #Yes, average distance to coastline is 2.68 km away, likely due to shipping concerns 

*   Do launch sites keep certain distance away from cities?
        #No, average distance to a city is 15.45 km away,  likely due to noise complaints concerns

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
